In [102]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.api as sm
from statsmodels.stats.weightstats import _tconfint_generic
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline

<font color='green'>
### 1

1-stats.t.cdf(t, nu)

<font color='green'>
### 2

Признаки устроены так, что μ1<μ2 в принципе невозможно

<font color='green'>
### 3

- Проблема заключается в том, что не существует точного способа сравнить средние двух выборок с неизвестными дисперсиями
- Приближённое решение проблемы Беренца-Фишера достаточно точно при n1=n2

<font color='green'>
### 4

In [2]:
mu0 = 9.5
sigma = 0.4
n = 160
X_mean = 9.57
def Z(mu0, sigma, X_mean, n):
    return (X_mean-mu0) / (sigma / np.sqrt(n))
z = Z(mu0, sigma, X_mean, n)
p = 2*(1-stats.norm.cdf(abs(z)))
print p

0.0268566955075


<font color='green'>
### 5

- Скорее всего нет: стандартное отклонение уровня в популяции намного больше полученного различия между средними

<font color='green'>
### 6

In [3]:
df_diamonds = pd.read_csv('diamonds.txt', sep='\t')
price = df_diamonds.price
del df_diamonds['price']

In [4]:
print len(price)

53940


In [5]:
df_diamonds.head()

,carat,depth,table,x,y,z
0,0.23,61.5,55.0,3.95,3.98,2.43
1,0.21,59.8,61.0,3.89,3.84,2.31
2,0.23,56.9,65.0,4.05,4.07,2.31
3,0.29,62.4,58.0,4.20,4.23,2.63
4,0.31,63.3,58.0,4.34,4.35,2.75


In [6]:
df_diamonds.tail()

,carat,depth,table,x,y,z
53935,0.72,60.8,57.0,5.75,5.76,3.50
53936,0.72,63.1,55.0,5.69,5.75,3.61
53937,0.70,62.8,60.0,5.66,5.68,3.56
53938,0.86,61.0,58.0,6.15,6.12,3.74
53939,0.75,62.2,55.0,5.83,5.87,3.64


In [7]:
df_diamonds = (df_diamonds - df_diamonds.mean()) / df_diamonds.std()

In [8]:
df_diamonds.head()

,carat,depth,table,x,y,z
0,-1.198157,-0.174090,-1.099662,-1.587823,-1.536181,-1.571115
1,-1.240350,-1.360726,1.585514,-1.641310,-1.658759,-1.741159
2,-1.198157,-3.384987,3.375631,-1.498677,-1.457382,-1.741159
3,-1.071577,0.454129,0.242926,-1.364959,-1.317293,-1.287708
4,-1.029384,1.082348,0.242926,-1.240155,-1.212227,-1.117663


In [9]:
df_diamonds['price'] = price

In [10]:
df_diamonds.head()

,carat,depth,table,x,y,z,price
0,-1.198157,-0.174090,-1.099662,-1.587823,-1.536181,-1.571115,326
1,-1.240350,-1.360726,1.585514,-1.641310,-1.658759,-1.741159,326
2,-1.198157,-3.384987,3.375631,-1.498677,-1.457382,-1.741159,327
3,-1.071577,0.454129,0.242926,-1.364959,-1.317293,-1.287708,334
4,-1.029384,1.082348,0.242926,-1.240155,-1.212227,-1.117663,335


In [11]:
X = df_diamonds[['carat', 'depth', 'table', 'x', 'y', 'z']].as_matrix()
y = df_diamonds[['price']].as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
diff_lr = abs(y_pred_lr - y_test)

In [13]:
print diff_lr.mean()

890.376400429


In [14]:
rf = RandomForestRegressor(random_state=1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test).reshape((X_test.shape[0], 1))
diff_rf = abs(y_pred_rf - y_test)

C:\Users\evasiliev\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [24]:
print diff_lr.flatten()

[  214.42235621  1016.46053685    41.87871693 ...,  2171.49180326
   103.82818096   159.32529706]


In [73]:
diff_means = diff_lr.mean() - diff_rf.mean()

In [98]:
t_stat = ttest_ind(diff_lr.flatten(), diff_rf.flatten(), equal_var=False)

In [99]:
t_stat.pvalue

5.3193154765534851e-10

<font color='green'>
### 7

In [107]:
diff = (diff_lr - diff_rf).flatten()
print stats.t.interval(0.95, len(diff)-1, loc=np.mean(diff), scale=stats.sem(diff))
print sm.DescrStatsW(diff.flatten()).tconfint_mean()

(74.429426522460119, 100.82097121452378)
(74.429426522460105, 100.82097121452377)
